In [1]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image 
import os
import glob
import pickle
import pandas as pd
import csv
from pathlib import Path
from IPython.display import HTML
from os import listdir, mkdir

from IPython import display

import tensorflow as tf
from keras.models import load_model
from keras import layers 
from keras.layers import Concatenate,Input, Dense, Activation, ZeroPadding2D, \
BatchNormalization, Flatten, Conv2D 
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout, GlobalMaxPooling2D, \
GlobalAveragePooling2D 
from keras.models import Model, Sequential 
from keras.preprocessing import image 
from keras.utils import layer_utils 
from keras.utils.data_utils import get_file 
from keras.applications.imagenet_utils import preprocess_input 
from sklearn.metrics import roc_auc_score, roc_curve, auc
import winsound

In [2]:
from platform import python_version

print(python_version())

3.8.8


In [3]:
path0 = "D:/cancer de pele/" 

In [4]:
doen = list(['akiec' , 'bcc' , 'mel' , 'nv' , 'bkl','df', 'vasc'])

In [5]:
dados        = pd.read_csv(path0+"HAM10000_metadata.csv")
dados.head(15)

,lesion_id,image_id,dx,dx_type,age,sex,localization
0,HAM_0000118,ISIC_0027419,bkl,histo,80.0,male,scalp
1,HAM_0000118,ISIC_0025030,bkl,histo,80.0,male,scalp
2,HAM_0002730,ISIC_0026769,bkl,histo,80.0,male,scalp
3,HAM_0002730,ISIC_0025661,bkl,histo,80.0,male,scalp
4,HAM_0001466,ISIC_0031633,bkl,histo,75.0,male,ear
5,HAM_0001466,ISIC_0027850,bkl,histo,75.0,male,ear
6,HAM_0002761,ISIC_0029176,bkl,histo,60.0,male,face
7,HAM_0002761,ISIC_0029068,bkl,histo,60.0,male,face
8,HAM_0005132,ISIC_0025837,bkl,histo,70.0,female,back
9,HAM_0005132,ISIC_0025209,bkl,histo,70.0,female,back


In [6]:
id_img_unique = pd.unique(dados['image_id'])
print("Número de imagens únicas: ", len(id_img_unique))

Número de imagens únicas:  10015


In [7]:
id_class_unique = pd.unique(dados['dx'])
numClasses = len(id_class_unique)

print("Número de classes: ", numClasses)
print("São elas: \n", doen)

Número de classes:  7
São elas: 
 ['akiec', 'bcc', 'mel', 'nv', 'bkl', 'df', 'vasc']


In [8]:
os.chdir('D:/cancer de pele/data_pickle/')
files = glob.glob('*.pckl')
print(files[0][:-5])

ISIC_0024306


In [9]:
files[103][:-5]

'ISIC_0024409'

In [10]:
len(files)

10015

In [11]:
lista = []
idx = 0
for k in files:
    x = [0]*8
    k =  k[:-5]
    x[0] = k
    res = dados.loc[(dados['image_id'] == k)]
    for i in res['dx']:
        j = int(doen.index(i) + 1)
        x[j] = 1
    lista.append(x)
    idx += 1
    if (idx%1000==0):
        print(idx)

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000


In [12]:
print(lista[:123])

[['ISIC_0024306', 0, 0, 0, 1, 0, 0, 0], ['ISIC_0024307', 0, 0, 0, 1, 0, 0, 0], ['ISIC_0024308', 0, 0, 0, 1, 0, 0, 0], ['ISIC_0024309', 0, 0, 0, 1, 0, 0, 0], ['ISIC_0024310', 0, 0, 1, 0, 0, 0, 0], ['ISIC_0024311', 0, 0, 0, 1, 0, 0, 0], ['ISIC_0024312', 0, 0, 0, 0, 1, 0, 0], ['ISIC_0024313', 0, 0, 1, 0, 0, 0, 0], ['ISIC_0024314', 0, 0, 0, 1, 0, 0, 0], ['ISIC_0024315', 0, 0, 1, 0, 0, 0, 0], ['ISIC_0024316', 0, 0, 0, 1, 0, 0, 0], ['ISIC_0024317', 0, 0, 0, 1, 0, 0, 0], ['ISIC_0024318', 0, 0, 0, 0, 0, 1, 0], ['ISIC_0024319', 0, 0, 0, 1, 0, 0, 0], ['ISIC_0024320', 0, 0, 0, 1, 0, 0, 0], ['ISIC_0024321', 0, 0, 0, 1, 0, 0, 0], ['ISIC_0024322', 0, 0, 0, 1, 0, 0, 0], ['ISIC_0024323', 0, 0, 1, 0, 0, 0, 0], ['ISIC_0024324', 0, 0, 0, 0, 1, 0, 0], ['ISIC_0024325', 0, 0, 0, 1, 0, 0, 0], ['ISIC_0024326', 0, 0, 0, 1, 0, 0, 0], ['ISIC_0024327', 0, 0, 0, 1, 0, 0, 0], ['ISIC_0024328', 0, 0, 0, 1, 0, 0, 0], ['ISIC_0024329', 1, 0, 0, 0, 0, 0, 0], ['ISIC_0024330', 0, 0, 0, 0, 0, 1, 0], ['ISIC_0024331', 0, 1, 0

In [13]:
colLabels = ['image_id'] + doen
print(colLabels)

dataf = pd.DataFrame(lista, columns=colLabels)
dataf.to_csv('C:/Users/alex_/Desktop/cancer-targetsTESTE.csv')

['image_id', 'akiec', 'bcc', 'mel', 'nv', 'bkl', 'df', 'vasc']


In [ ]:
dataf.head()

In [ ]:
len(dataf)

In [ ]:
os.chdir(path0+"data_pckl/")
files=glob.glob("*.pckl")

In [ ]:
files[0][:-5]

In [ ]:
dataf.iloc[3000:3001,:]

In [ ]:
Y = np.zeros((len(files),7))
for i in range(len(files)):
  for j in range(len(dataf)): 
     if files[i][:-5] == dataf.iloc[j,0]:
        for k in range(7):    
          Y[i,k] = dataf.iloc[j][k+1]

In [ ]:
Y

In [ ]:
### Load data
f = open(files[0], 'rb')
X = pickle.load(f)

In [ ]:
plt.imshow(X,cmap=plt.cm.bone)

In [ ]:
fig = plt.figure(figsize=(10,12))
X   = np.zeros((len(files),450,600,3))
for j in range(0,len(files)):
   f = open(files[j], 'rb')
   X[j] = pickle.load(f)

In [ ]:
Path     =  path0 + "0-Data.pckl"


In [ ]:
f = open(Path, 'wb')
pickle.dump([X,Y], f)
f.close()

In [ ]:
f = open(Path, 'rb')
[X,Y] = pickle.load(f)

In [ ]:
plt.imshow(X[0,:,:,2])
print(Y[0])

In [ ]:
def KerasModel(input_shape):  
    NumClass = 7
    X_input  = Input(input_shape)
     
    # Zero-Padding: pads the border of X_input with zeroes
    X = ZeroPadding2D((3, 3))(X_input)

    # CONV -> BN -> RELU Block applied to X
    X = Conv2D(40, (3, 3), strides = (1, 1), name = 'conv0')(X)    
    X = BatchNormalization(axis = 3, name = 'bn0')(X)
    X = Activation('relu')(X)

    # MAXPOOL
    X = MaxPooling2D((2, 2), name='max_pool')(X)
    
    ##
    X = Conv2D(24, (3, 3), strides = (1, 1), name = 'conv1')(X)    
    X = BatchNormalization(axis = 3, name = 'bn1')(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((2, 2), name='max_pool1')(X)  
    ##
    
    ##
    X = Conv2D(16, (3, 3), strides = (1, 1), name = 'conv2')(X)    
    X = BatchNormalization(axis = 3, name = 'bn2')(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((2, 2), name='max_pool2')(X)  
    ##    

    ##   New layer
    X = Conv2D(8, (3, 3), strides = (1, 1), name = 'conv3')(X)    
    X = BatchNormalization(axis = 3, name = 'bn3')(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((2, 2), name='max_pool3')(X)  
    ##    


    # FLATTEN X (means convert it to a vector) + FULLYCONNECTED
    X = Flatten()(X)
    X = Dense(NumClass, activation='sigmoid', name='fc')(X)

    # Create model. This creates your Keras model instance, you'll use this instance to train/test the model.
    model = Model(inputs = X_input, outputs = X, name='KerasModel')
    
    return model 

In [ ]:
# Construct the model
ModelPulm = KerasModel(X[0].shape) # input dimention
# Run the model
ModelPulm.compile(loss='binary_crossentropy',optimizer='Adam', metrics=['accuracy'])

In [ ]:
history = ModelPulm.fit( x=X, y=Y, batch_size=2**6, epochs=5) ## batch = 2**6
plt.plot(history.history['accuracy'])
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
Pegar aleatoriamente
Resize nas img
ver data do tensorflow lite